<a href="https://colab.research.google.com/github/Lada496/CrewAI-Projects/blob/main/Crew_AI_Job_Interview_Prep_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')

In [ ]:
!pip install 'crewai[tools]' -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 1.1.6 requires openai<3.0.0,>=1.109.1, but you have openai 1.83.0 which is incompatible.


In [ ]:
!pip install langchain_openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 1.12.0 requires openai<2.0.0,>=1.70.0, but you have openai 2.14.0 which is incompatible.
crewai 1.7.2 requires openai~=1.83.0, but you have openai 2.14.0 which is incompatible.


In [ ]:
# Import libraries
from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
import os
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource

In [ ]:
# Set the API keys as environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
# Define the LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.5)

# Get the data
- Interviewer
- company
- job position
- job description

In [ ]:
# Store the data
interviewer = input("Enter the name of the interviewer: ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

Enter the name of the interviewer: Dr. Alex Chen
Enter the name of the company: Aurora AI Labs
Enter the job position: Machine Learning Engineer Intern (NLP Focus)
Enter the job description: We are looking for a Machine Learning Engineer Intern with a strong interest in Natural Language Processing (NLP) to join our AI team.  In this role, you will work on building, training, and evaluating machine learning models for text classification and regression tasks. You will collaborate closely with software engineers and data scientists to improve model performance, data pipelines, and deployment workflows.  Responsibilities: - Build and train machine learning models using TensorFlow or PyTorch - Work with NLP techniques such as tokenization, embeddings (GloVe, Word2Vec, Transformer-based models), and sequence modeling - Evaluate models using appropriate metrics (MAE, accuracy, F1-score) - Perform data preprocessing, cleaning, and augmentation - Write clean, well-documented, and testable code

# Interviewer AI Agent

In [ ]:
# Create the interviewer AI agent
interviewer_agent = Agent(
    role=f"You are {interviewer}, employed at {company}, and hiring for the job position {job_position}.",
    goal=f"You help the user prepare for the job interview for {job_position} with the following description: {job_description}.",
    backstory="""
      Role: Senior Machine Learning Engineer at Aurora AI Labs
      Background: Dr. Chen specializes in NLP systems for educational technology and has experience deploying ML models at scale. He focuses heavily on understanding candidates’ reasoning, model evaluation choices, and ability to explain technical decisions clearly.
    """,
    llm=llm,
    max_iterations=1
)

In [ ]:
# Define the interview prep task
interview_prep_task = Task(
    description=f"""
    Interview the user for the job {job_position} with the following description: {job_description}.
    """,
    expected_output=f"""
    ask only ONE question to the user that is relevant for the job {job_position} that has not been asked before, and END interview after receiving one answer.
    """,
    agent=interviewer_agent,
    human_input=True,
)

In [ ]:
# Build the AI interview coach agent
coach_agent = Agent(
    role="Interview Coach",
    goal=f"I help the user prepare for the job interview for {job_position} with the following description: {job_description}.",
    backstory="Your are an expert in job interviews",
    llm=llm
)

In [ ]:
# Define the Coaching Task
coaching_task = Task(
    description="""Give feedback to the user on its answer""",
    expected_output="""Give feedback to the user on its answer with bulletpoints on what was good, bad, and how to take it to the next level""",
    agent=coach_agent,
    context=[interview_prep_task]
)

# AI Crew

In [ ]:
# Create an empty list for questions
interview_questions = []

# Store each question
# interview_questions.append(result.tasks_output[0].raw)
# interview_contet = ' '.join(interview_questions)
# string_source = StringKnowledgeSource(content = interview_contet)

In [ ]:
crew = Crew(
    agents=[interviewer_agent, coach_agent],
    tasks=[interview_prep_task, coaching_task],
    verbose=True,
    process=Process.sequential,
    memory=True,
    knowledge_sources=[string_source]
)
result = crew.kickoff()

# Store each question
interview_questions.append(result.tasks_output[0].raw)
interview_contet = ' '.join(interview_questions)
string_source = StringKnowledgeSource(content = interview_contet)

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: fd8008f1-abf5-43b3-ae7c-64a31d347a23                                    │
│  Tool Args:                                                                  │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b7874acc-8a27-46d8-b3f9-42cd734d1102
    Status: Executing Task...╭────────────────────────────── 🤖 Agent Started ──────────────────────────────╮
│                                                                              │
│  Agent: You are 

In [ ]:
# Check output
display(Markdown(result.tasks_output[1].raw))

- **Good Points:**
  - Demonstrates practical experience with key NLP techniques such as tokenization, GloVe embeddings, and LSTM for sequence modeling, which are relevant to the role.
  - Shows understanding of model evaluation metrics by using Mean Absolute Error and exact-match accuracy, aligning with the job's requirements.
  - Highlights a proactive approach to improving model performance by analyzing high-error essays, which shows a problem-solving mindset.
  - The project experience shared is directly aligned with the responsibilities of the Machine Learning Engineer Intern position, making it a strong fit.

- **Areas for Improvement:**
  - While the response is strong, it could benefit from more details on collaborative experiences, such as working with software engineers or data scientists, to emphasize teamwork skills.
  - Consider mentioning any experience with Git or collaborative development practices to address the requirement for experience with version control systems.
  - Highlight any experience with integrating models into applications, which is part of the job responsibilities.

- **Next Level Suggestions:**
  - Discuss any experience with pre-trained language models or TensorFlow Hub, as these are noted as "nice to have" skills in the job description.
  - If applicable, mention familiarity with REST APIs or GraphQL, as this would enhance your application by aligning with the "nice to have" requirements.
  - Consider sharing an example of writing clean, well-documented, and testable code, perhaps through a specific instance of test-driven development, to showcase your coding practices.
  - Emphasize any communication skills or experiences in collaborative settings that demonstrate your ability to work effectively in a team environment.

In [ ]:
#